In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# retrieve BOW vocab using usual sklearn fit method
texts = ["hello", "world", "this", "is", "me"]
cv = CountVectorizer()

cv.fit(texts)
cv.vocabulary_
# {'hello': 0, 'world': 4, 'this': 3, 'is': 1, 'me': 2}


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# explicitly define vocabulary at point of instantiation
texts = ["hello", "world", "this", "is", "me"]
cv = CountVectorizer(vocabulary=texts)  # err


In [ ]:
cv.vocabulary_
# ?cv.set_params


In [4]:
import pandas as pd

df = pd.read_csv("/Users/samhardy/Desktop/otso-nlp-util/tests/data/test_dataset.csv")


In [58]:
query = """(super AND amg super) OR (super AND australian catholic super) OR (super AND australiansuper) OR (super AND aware super) OR (super AND cbus) OR (super AND caresuper) OR (super AND colonial first estate) OR (super AND eiss super) OR (super AND energysuper) OR (super AND equipsuper) OR (super AND firststate super growth) OR (super AND future super) OR (super AND guild super) OR (super AND hesta super fund) OR (super AND hostplus) OR (super AND intrust) OR (super AND lgiasuper)                  20
(banking AND suncorp) OR (banking AND teachers mutual bank) OR (banking AND westpac)"""


df = (
    pd.read_csv(
        "/Users/samhardy/Downloads/otso-data-bruce/bruce_base_modelling/data/reddit/2021_07_27/transformed_data/2021_reddit_all_industries.csv"
    )
    .query('subreddit == "AusFinance"')
    .pipe(lambda x: x[x["query"].str.contains("(super AND amg super)")])
    .pipe(lambda x: x[~x.body.isna()])
    .drop_duplicates("body")
    .pipe(lambda x: x[x.body.apply(lambda y: len(y) <= 100)])
    .pipe(lambda x: x[x.body.apply(lambda y: len(y) >= 20)])
    .sample(n=100, random_state=42)
    .reset_index(drop=True)
)


/var/folders/wc/l2_1jymx3m398ydltqtl1nf80000gn/T/ipykernel_89220/3565190106.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  .pipe(lambda x: x[x["query"].str.contains("(super AND amg super)")])


In [5]:
from clear_bow.classifier import DictionaryClassifier

super_dict = {
    "regulation": ["asic", "government", "federal", "tax"],
    "contribution": ["contribution", "concession", "personal", "after tax", "10%", "10.5%"],
    "covid": ["covid", "lockdown", "downturn", "effect"],
    "retirement": ["retire", "house", "annuity", "age"],
    "fund": ["unisuper", "aus super", "australian super", "sun super", "qsuper", "rest", "cbus"],
}
dc = DictionaryClassifier(label_dictionary=super_dict)  # multi-class by default
dc.predict_single("A 10% contribution is not enough for a well balanced super fund!")
# {'regulation': 0.0878,
#  'contribution': 0.6488,
#  'covid': 0.0878,
#  'retirement': 0.0878,
#  'fund': 0.0878}


dc.predict_batch(
    [
        "A 10% contribution is not enough for a well balanced super fund!",
        "Australian government should stay the hell out of my retirement plans!",
    ]
)
# [{'regulation': 0.0878,
#   'contribution': 0.6488,
#   'covid': 0.0878,
#   'retirement': 0.0878,
#   'fund': 0.0878},
#  {'regulation': 0.3222,
#   'contribution': 0.1185,
#   'covid': 0.1185,
#   'retirement': 0.3222,
#   'fund': 0.1185}]


{'regulation': 0.0878,
 'contribution': 0.6488,
 'covid': 0.0878,
 'retirement': 0.0878,
 'fund': 0.0878}

[{'regulation': 0.0878,
  'contribution': 0.6488,
  'covid': 0.0878,
  'retirement': 0.0878,
  'fund': 0.0878},
 {'regulation': 0.3222,
  'contribution': 0.1185,
  'covid': 0.1185,
  'retirement': 0.3222,
  'fund': 0.1185}]

In [71]:
preds = pd.DataFrame(list(df.body.apply(dc.predict_single)))

for e in super_dict.keys():
    mask = preds[e] > 0.3
    df[mask].head(3).body.tolist()

['And before Asic politely reminded them they cannot provide financial advice',
 'Taxes are not fees. They are a federal government tax.',
 "That's some other government's problem."]

['Mate you get 15% tax on concessional contributions in super.\n\nSo no surprise it adds up to over 15%.',
 'Personal insurance and health insurance cover different things too.',
 'Is the ‘income’ figure for this before or after tax?']

['Message from Covid 19: “don’t touch your face, don’t touch your super”',
 "It would be better than having an imaginary jetski that I can't use after the lockdown"]

['Not really unless you will retire fairly soon',
 'If you need a house, you need a house. I would suggest trying to top it back up when you can.',
 'Message from Covid 19: “don’t touch your face, don’t touch your super”']

["I'm not a Financial expert but Australian super are one of the bigger better options I guess",
 'No but considering moving I’m with unisuper at the moment their a closed fund',
 'Sun Super all the way']

In [74]:
x = [
    [
        "And before Asic politely reminded them they cannot provide financial advice",
        "Taxes are not fees. They are a federal government tax.",
        "That's some other government's problem.",
    ],
    [
        "Mate you get 15% tax on concessional contributions in super.\n\nSo no surprise it adds up to over 15%.",
        "Personal insurance and health insurance cover different things too.",
        "Is the ‘income’ figure for this before or after tax?",
    ],
    [
        "Message from Covid 19: “don’t touch your face, don’t touch your super”",
        "It would be better than having an imaginary jetski that I can't use after the lockdown",
    ],
    [
        "Not really unless you will retire fairly soon",
        "If you need a house, you need a house. I would suggest trying to top it back up when you can.",
        "Message from Covid 19: “don’t touch your face, don’t touch your super”",
    ],
    [
        "I'm not a Financial expert but Australian super are one of the bigger better options I guess",
        "No but considering moving I’m with unisuper at the moment their a closed fund",
        "Sun Super all the way",
    ],
]


example_docs = [item for sublist in x for item in sublist]


In [77]:
import random

random.shuffle(example_docs)

example_docs

["I'm not a Financial expert but Australian super are one of the bigger better options I guess",
 'Mate you get 15% tax on concessional contributions in super.\n\nSo no surprise it adds up to over 15%.',
 'Is the ‘income’ figure for this before or after tax?',
 'Message from Covid 19: “don’t touch your face, don’t touch your super”',
 "It would be better than having an imaginary jetski that I can't use after the lockdown",
 'Not really unless you will retire fairly soon',
 "That's some other government's problem.",
 'Sun Super all the way',
 'Message from Covid 19: “don’t touch your face, don’t touch your super”',
 'And before Asic politely reminded them they cannot provide financial advice',
 'Personal insurance and health insurance cover different things too.',
 'If you need a house, you need a house. I would suggest trying to top it back up when you can.',
 'Taxes are not fees. They are a federal government tax.',
 'No but considering moving I’m with unisuper at the moment their a c

In [62]:
mask

0     False
1     False
2      True
3     False
4     False
      ...  
95    False
96    False
97     True
98    False
99    False
Name: fund, Length: 100, dtype: bool